In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: ml-studio-labs
Azure region: southeastasia
Subscription id: 5e4d75b9-5b13-49fb-8306-ae971a3c14b1
Resource group: azure-mlops


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "ml-studio-clust"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_v2',
                                                           max_nodes=1)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.5, 2.0),
    "max_iter": choice(100, 200, 300)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
arguments = [
    "--C",
    1.0,
    "--max_iter",
    100,
]

src = ScriptRunConfig(source_directory=".",
                      script="./training/train.py",
                      arguments=arguments,
                      environment=sklearn_env,
                      compute_target=compute_target)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    run_config=src,
    policy=policy,
    max_total_runs=20,
    max_duration_minutes=30
)

In [4]:
from azureml.widgets import RunDetails
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = Experiment(ws, exp.name).submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [5]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_7821a912-2f05-42a1-b221-d9392a15ea85
Web View: https://ml.azure.com/runs/HD_7821a912-2f05-42a1-b221-d9392a15ea85?wsid=/subscriptions/5e4d75b9-5b13-49fb-8306-ae971a3c14b1/resourcegroups/azure-mlops/workspaces/ml-studio-labs&tid=f336fb5b-9257-44b3-a041-3897edf080c9

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-03-10T08:13:55.544466][API][INFO]Experiment created<END>\n""<START>[2022-03-10T08:13:56.341182][GENERATOR][INFO]Trying to sample '20' jobs from the hyperparameter space<END>\n""<START>[2022-03-10T08:13:57.176178][GENERATOR][INFO]Successfully sampled '20' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_7821a912-2f05-42a1-b221-d9392a15ea85
Web View: https://ml.azure.com/runs/HD_7821a912-2f05-42a1-b221-d9392a15ea85?wsid=/subscriptions/5e4d75b9-5b13-49fb-8306-ae971a3c14b1/resourcegroups/azure-mlops/workspaces/ml-studio-labs&tid=f336fb5b-9257-44b3-a041-3897edf080c9



{'runId': 'HD_7821a912-2f05-42a1-b221-d9392a15ea85',
 'target': 'ml-studio-clust',
 'status': 'Completed',
 'startTimeUtc': '2022-03-10T08:13:55.327995Z',
 'endTimeUtc': '2022-03-10T08:45:09.573363Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '15fe3a7a-38e9-4f7a-881b-4c0ea28a456e',
  'user_agent': 'python/3.8.1 (Linux-5.4.0-1068-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': 'infinite_space_size',
  'score': '0.9128983308042489',
  'best_child_run_id': 'HD_7821a912-2f05-42a1-b221-d9392a15ea85_10',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstudiolabs9848758000.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_7821a912-2f05-42a1-b22

In [13]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(f'Best run ID: {best_run.id}')
print(f'Best metrics: \n{best_run.get_metrics()}')

best_model = best_run.register_model(model_name='bankmarketing-model', model_path='outputs/model.joblib')

Best run ID: HD_7821a912-2f05-42a1-b221-d9392a15ea85_10
Best metrics: 
{'Regularization Strength:': 1.4825446427827171, 'Max iterations:': 300, 'Accuracy': 0.9128983308042489}


# AutoML

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(url)

In [6]:
import pandas as pd
from training.train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
full_data = pd.concat([x, y], axis=1)

In [7]:
data_dir = "data"

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

data_path = data_dir + "/bankmarketing_train.csv"
full_data.to_csv(data_path, index=False)
print("Data written to local folder.")

Data written to local folder.


In [8]:
!ls -R {data_dir}

data:
bankmarketing_train.csv


In [9]:
default_store = ws.get_default_datastore()

default_store.upload_files([data_path], 
                           target_path="data", 
                           overwrite=False, 
                           show_progress=True)

print("Upload calls completed.")

"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Target already exists. Skipping upload for data/bankmarketing_train.csv
Uploaded 0 files
Upload calls completed.


In [10]:
# Create and register datasets
from azureml.core import Dataset
full_data = Dataset.Tabular.from_delimited_files(default_store.path(data_dir))
full_ds = full_data.register(ws, 'bankmarketing-train')

print("Dataset is registered.")

Dataset is registered.


In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=full_ds,
    label_column_name="y",
    n_cross_validations=2,
    blocked_models=["XGBoostClassifier"],
    compute_target=compute_target,
    verbosity=logging.INFO)

In [12]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on ml-studio-clust with default configuration
Running on remote compute: ml-studio-clust


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_a9f541b9-a3f5-4e6b-a16d-88eb9efbdba7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

In [23]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, best_automl_model = automl_run.get_output()

print(f"Best AutoML Run: \n{best_automl_run}")
print("================================================================")
print(f"Best AutoML Model: \n{best_automl_model}")

model_path = "outputs/automl_model.joblib"
joblib.dump(best_automl_model, model_path)

best_automl_run.register_model(model_name="bankmarketing-model", model_path="outputs")

Best AutoML Run: 
Run(Experiment: udacity-project,
Id: AutoML_a9f541b9-a3f5-4e6b-a16d-88eb9efbdba7_27,
Type: azureml.scriptrun,
Status: Completed)
Best AutoML Model: 
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('19', Pipeline(memory=None, steps=[('truncatedsvdwrapper', TruncatedSVD

Model(workspace=Workspace.create(name='ml-studio-labs', subscription_id='5e4d75b9-5b13-49fb-8306-ae971a3c14b1', resource_group='azure-mlops'), name=bankmarketing-model, id=bankmarketing-model:2, version=2, tags={}, properties={})

In [ ]:
# cluster_cleanup
compute_target.delete()